In [0]:
#Note to maintainer - Please do not import all functions of pyspark module, masks default pandas functions. If you need anything specifc, import by name rather than using * 

#Python
import pandas as pd
import os,numpy as np
import glob2
import datetime

#Pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,when,lit
from pyspark.sql import functions as F

In [0]:
#Create BRAND AND PTD Lookup DF
START_DATE = datetime.date(2019, 9, 30)
DATE_LIST = [( START_DATE + datetime.timedelta(days=i)) for i  in range(0,720,7)]

LOOKUP_DF = pd.DataFrame(columns = ['DATE'])
LOOKUP_DF['DATE'] = DATE_LIST
LOOKUP_DF['DATE'] = pd.to_datetime(LOOKUP_DF['DATE'])


#Convert to date  and get week number from 
G_CAL = pd.read_csv("/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/GCAL.csv")
G_CAL = G_CAL[['GCALDAY','GMARSWINP','GMARSWINP___T','GMARSPINY','GMARSPINY___T','GCALYEAR']]
G_CAL['DATE'] = pd.to_datetime(G_CAL['GCALDAY'],format='%Y%m%d')
#Merge with G_CAL
LOOKUP_DF = pd.merge(LOOKUP_DF,G_CAL,how='left')

LOOKUP_DF.loc[LOOKUP_DF['GMARSWINP___T']=='W1', 'BRAND_FILE_PERIOD'] = (LOOKUP_DF['GMARSPINY'] - 2)
LOOKUP_DF.loc[LOOKUP_DF['GMARSWINP___T']!='W1', 'BRAND_FILE_PERIOD'] = (LOOKUP_DF['GMARSPINY'] - 1)
LOOKUP_DF['BRAND_FILE_PERIOD'] = pd.to_numeric(LOOKUP_DF['BRAND_FILE_PERIOD'], downcast = 'integer')
LOOKUP_DF.loc[LOOKUP_DF['BRAND_FILE_PERIOD']==0,'BRAND_FILE_PERIOD']=13
LOOKUP_DF.loc[LOOKUP_DF['BRAND_FILE_PERIOD']==-1,'BRAND_FILE_PERIOD']=12
# LOOKUP_DF.head(10)
# LOOKUP_DF.loc[LOOKUP_DF['GCALYEAR']==2020]


In [0]:
#Get details about current date,day,week,period,month,year,etc
CURR_DATE = datetime.date.today()
# CURR_DATE = datetime.date(2020, 11,10)

TS_OBJ_CURR_DATE = pd.Timestamp(LOOKUP_DF.loc[LOOKUP_DF['DATE']<=CURR_DATE, 'DATE'].tail(1).values[0])
CURR_DATE = TS_OBJ_CURR_DATE.to_pydatetime().date()

CURR_DATE_STRING = str(CURR_DATE.year) + CURR_DATE.strftime('%m') + CURR_DATE.strftime('%d')

print("Current Date String", CURR_DATE_STRING)

#Calulate last saturday
idx = (CURR_DATE.weekday() + 1) % 7 

PTD_FILE_DATE = CURR_DATE - datetime.timedelta(7+idx-6)
print("PTD_FILE_DATE",PTD_FILE_DATE)

PTD_PERIOD = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GMARSPINY'].values[0]   
PTD_WEEK = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GMARSWINP___T'].values[0]   
PTD_YEAR = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GCALYEAR'].values[0]   

print('PTD_PERIOD',PTD_PERIOD)
print('PTD_WEEK',PTD_WEEK)


Current Date String 20201130
PTD_FILE_DATE 2020-11-28
PTD_PERIOD 12
PTD_WEEK W4

In [0]:
#PTD File - Overall
PTD_FILE_NAME = '/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2020/Markets_NSV_PTD.xlsx'
print("PTD_FILE_NAME: ", PTD_FILE_NAME)

# #LE File
# YEE_FILE_DIR_PATH = "/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/" + str(datetime.datetime.today().year)
# YEE_FILE_DIR_PATH = glob2.glob(YEE_FILE_DIR_PATH + "/*YEE*.xlsx")
# YEE_FILE_DIR_IDX = [int((i.split("YEE P"))[1].split(" Blob")[0])  for i in YEE_FILE_DIR_PATH]
# CURR_PERIOD_YEE_FILE_NAME = YEE_FILE_DIR_PATH[YEE_FILE_DIR_IDX.index(max(YEE_FILE_DIR_IDX))]

CURR_PERIOD_YEE_FILE_NAME='/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2020/NSV KIND.xlsx'

print("CURR_PERIOD_YEE_FILE_NAME ",CURR_PERIOD_YEE_FILE_NAME)

try:
  LAST_PERIOD_YEE_FILE_NAME = YEE_FILE_DIR_PATH[YEE_FILE_DIR_IDX.index(max(YEE_FILE_DIR_IDX)-1)]
  print("LAST_PERIOD_YEE_FILE_NAME ",LAST_PERIOD_YEE_FILE_NAME)
except:
  pass


#Brand Data File

CURR_PERIOD_BRAND_FILE_NAME = '/dbfs/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/WATERFALL_TABLE.csv'
print("CURR_PERIOD_BRAND_FILE_NAME ",CURR_PERIOD_BRAND_FILE_NAME)


#Replan File
REPLAN_FILE_NAME = ("/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/" + str(PTD_FILE_DATE.year) + "/REPLAN 2020.xlsx")
print("REPLAN_FILE_NAME ", REPLAN_FILE_NAME)


#Internal Mappin gFIle
INTERNAL_MAPPING_FILE_NAME = '/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/INTERNAL/KIND_INTERNAL_MAPPING.xlsx'
print('INTERNAL_MAPPING_FILE_NAME   ', INTERNAL_MAPPING_FILE_NAME)

PTD_FILE_NAME: /dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2020/Markets_NSV_PTD.xlsx
CURR_PERIOD_YEE_FILE_NAME /dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2020/NSV KIND.xlsx
CURR_PERIOD_BRAND_FILE_NAME /dbfs/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/P&L DASHBOARD/WATERFALL_TABLE.csv
REPLAN_FILE_NAME /dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2020/REPLAN 2020.xlsx
INTERNAL_MAPPING_FILE_NAME /dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/CONFIG_FILES/INTERNAL/KIND_INTERNAL_MAPPING.xlsx

In [0]:
PTD_OUTPUT_DF = pd.read_excel(PTD_FILE_NAME)

#Defining the schema to convert the Pandas dataframe into a spark dataframe
PTD_TABLE_SCHEMA = StructType([StructField("YEAR", StringType(), True)\
                               ,StructField("PERIOD", IntegerType(), True)\
                               ,StructField("WEEK", StringType(), True)\
                               ,StructField("KIND Markets Helios", StringType(), True)\
                               ,StructField("KIND Markets", StringType(), True)\
                               ,StructField("KIND Clusters", StringType(), True)\
                               ,StructField("KIND Regions", StringType(), True)\
                               ,StructField("NSV", DoubleType(), True)])

#Creating a spark dataframe
PTD_OUTPUT_TABLE = spark.createDataFrame(PTD_OUTPUT_DF,schema = PTD_TABLE_SCHEMA)


# # replacing Nan values with Null values
columns=PTD_OUTPUT_TABLE.columns
for column in columns:
    PTD_OUTPUT_TABLE = PTD_OUTPUT_TABLE.withColumn(column,F.when(F.isnan(F.col(column)),None).otherwise(F.col(column)))
    
PTD_OUTPUT_TABLE=PTD_OUTPUT_TABLE.filter(~(col("NSV")).isNull())
    
# Write Final dataframe
PTD_OUTPUT_TABLE.coalesce(1).write.mode('overwrite').option("nullValue", "").option("treatEmptyValuesAsNulls", "true").csv('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/WEEKLY TRACKER/PTD_OUTPUT_TABLE.csv',header=True)
CSV_DIR = 'dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/WEEKLY TRACKER/'
FILES = [str(x.name) for x in dbutils.fs.ls('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/WEEKLY TRACKER/PTD_OUTPUT_TABLE.csv')]
for i in range(len(FILES)):
  if(FILES[i].startswith('part')):
          dbutils.fs.cp(CSV_DIR  + 'PTD_OUTPUT_TABLE.csv/' + FILES[i],CSV_DIR + FILES[i])
          dbutils.fs.rm(CSV_DIR+'PTD_OUTPUT_TABLE.csv' , True)
          dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'PTD_OUTPUT_TABLE.csv')

In [0]:
# ## Check if the file contains current week's input
# max_year=PTD_OUTPUT_TABLE.agg({"YEAR":"max"}).collect()[0][0]
# max_period=PTD_OUTPUT_TABLE.filter(col("YEAR")==max_year).agg({"PERIOD":"max"}).collect()[0][0]
# max_week=PTD_OUTPUT_TABLE.filter((col("YEAR")==max_year)&(col("PERIOD")==max_period)).agg({"WEEK":"max"}).collect()[0][0]

# max_period,max_week

# print("Max Period Available",max_period)
# print("Max Week Available",max_week)

# if (max_period==PTD_PERIOD ) & (max_week==PTD_WEEK):
#   print("QC Check pass for PTD File")
# else:
#   import json
#   import urllib
#   # Replace this with the URI and API Key for your web service
#   body = str.encode(json.dumps(1))
#   url = "https://prod-35.eastus2.logic.azure.com:443/workflows/81c653b993394b23aa0c241d5dca3e9a/triggers/manual/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=93BgCu3jVJnKxtybkUdykEBasteXF4StHzZ42roOTKk"
#   headers = {'Content-Type':'application/json'}
#   req = urllib.request.Request(url,body, headers)
#   response = urllib.request.urlopen(req)
#   print("QC Check fail for PTD File")
#   FAIL

In [0]:
DF_PTD = pd.read_excel(PTD_FILE_NAME, usecols = ['KIND Markets Helios','NSV','PERIOD','WEEK','YEAR'])
DF_PTD = DF_PTD.groupby(by=['KIND Markets Helios','PERIOD','WEEK','YEAR'], as_index=False).agg('sum')


PTD_PERIOD = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GMARSPINY'].values[0]   
PTD_WEEK = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GMARSWINP___T'].values[0]   
PTD_YEAR = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GCALYEAR'].values[0]   

print('PTD_PERIOD',PTD_PERIOD)
print('PTD_WEEK',PTD_WEEK)
print('PTD_YEAR',PTD_YEAR)

# #Filter for period week and year and calculate PTD Values
DF_PTD = DF_PTD.loc[(DF_PTD['PERIOD']==PTD_PERIOD) & (DF_PTD['WEEK']==PTD_WEEK) & (DF_PTD['YEAR']==PTD_YEAR)]
DF_PTD = DF_PTD[['KIND Markets Helios','NSV']]
DF_PTD.columns = ['MARKET','PTD']
DF_PTD.head(20)
DF_PTD['PTD'] = DF_PTD['PTD'] 
DF_PTD.reset_index(drop=True, inplace = True)
DF_PTD.loc[len(DF_PTD)] = ['ROW TSA',0.0]
DF_PTD.loc[len(DF_PTD)] = ['DMEA',0.0]
DF_PTD.loc[len(DF_PTD)] = ['TSA',0.0]

# DF_PTD.MARKET.unique()
print('NSV',DF_PTD['PTD'].sum())

PTD_PERIOD 12
PTD_WEEK W4
PTD_YEAR 2020
NSV 5.84462613248

In [0]:
DF_PTD = pd.read_excel(PTD_FILE_NAME, usecols = ['KIND Markets Helios','NSV','PERIOD','WEEK','YEAR'])
DF_PTD['NSV'] = DF_PTD['NSV'].replace("Infinity", np.nan)
DF_PTD['NSV'] = DF_PTD['NSV'].replace("-Infinity", np.nan)

# DF_PTD['NSV Manual'] = DF_PTD['NSV Manual'].replace("Infinity", np.nan)
# DF_PTD['NSV Manual'] = DF_PTD['NSV Manual'].replace("-Infinity", np.nan)

# DF_PTD['NSV Manual'] = DF_PTD['NSV Manual'].astype(float)
DF_PTD['NSV'] = DF_PTD['NSV'].astype(float)

DF_PTD = DF_PTD.groupby(by=['KIND Markets Helios','PERIOD','WEEK','YEAR'], as_index=False).agg('sum')


PTD_PERIOD = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GMARSPINY'].values[0]   
PTD_WEEK = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GMARSWINP___T'].values[0]   
PTD_YEAR = G_CAL.loc[G_CAL['GCALDAY']==int((str(PTD_FILE_DATE.year) + PTD_FILE_DATE.strftime('%m') + PTD_FILE_DATE.strftime('%d'))), 'GCALYEAR'].values[0]   

print('PTD_PERIOD',PTD_PERIOD)
print('PTD_WEEK',PTD_WEEK)

#Filter for period week and year and calculate PTD Values
DF_PTD = DF_PTD.loc[(DF_PTD['PERIOD']==PTD_PERIOD) & (DF_PTD['WEEK']==PTD_WEEK) & (DF_PTD['YEAR']==PTD_YEAR)]
DF_PTD = DF_PTD[['KIND Markets Helios','NSV']]
DF_PTD.columns = ['MARKET','PTD']
DF_PTD['PTD'] = DF_PTD['PTD'] 
DF_PTD.reset_index(drop=True, inplace = True)
DF_PTD.loc[len(DF_PTD)] = ['TSA',0.0]
DF_PTD.loc[len(DF_PTD)] = ['DMEA',0.0]
DF_PTD.loc[len(DF_PTD)] = ['ROW TSA',0.0]


#2. Add PER LE FILE
COL_NUMBER_CURR_PERIOD = PTD_PERIOD
# CURR_PERIOD_LE_NAME = 'P' + str(CURRENT_PERIOD_NUMBER) + ' LE'
print(COL_NUMBER_CURR_PERIOD)
# print(CURR_PERIOD_LE_NAME)
CURR_PER_LE = pd.read_excel(CURR_PERIOD_YEE_FILE_NAME,parse_cols="A,P:AB",usecols=[0,COL_NUMBER_CURR_PERIOD] ,skiprows = 4, names = ['MARKET','PER LE'])
CURR_PER_LE=CURR_PER_LE.iloc[:33,:]
#CURR_PER_LE['PER LE'] = CURR_PER_LE['PER LE'].round(2)

# #Remove blank rows
CURR_PER_LE.dropna(subset=['MARKET'],inplace=True)


#Remove total rows 
CURR_PER_LE = CURR_PER_LE.loc[~CURR_PER_LE['MARKET'].isin(['KIND TOTAL','ROW','UK/IRE','CHINA/NEA','ROW (excl. TSA)'])]

#Standartize market names across YEE File to match helios names
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='UK', 'MARKET'] = 'KIND International Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='Hong Kong', 'MARKET'] = 'Hong Kong Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='USA', 'MARKET'] = 'KIND USA Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=="DESIGNATED CAT.","MARKET"] = 'KIND USA Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=="KIND USA","MARKET"] = 'KIND USA Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=="Gulf","MARKET"] = 'Gulf Market'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=="Central Europe","MARKET"] = 'Romania'


# #Merge
DF_PTD = pd.merge(DF_PTD, CURR_PER_LE,how='left')
# DF_PTD.head(10)

# #3. Difference $ PTD vs P6 LE
DF_PTD['ABS PTD vs PER LE'] = DF_PTD['PTD'] - DF_PTD['PER LE']
DF_PTD.head(10)


# #4. % Difference $ PTD vs P6 LE
DF_PTD['PER PTD vs PER LE'] = round(((DF_PTD['PTD'] / DF_PTD['PER LE'])),2)


#Replace all infinity and Nan with  0
DF_PTD = DF_PTD.replace(-np.Inf,np.nan)
DF_PTD = DF_PTD.replace(np.Inf,np.nan)

print('PER LE SUM', DF_PTD['PER LE'].sum())
print('PTD SUM', DF_PTD['PTD'].sum())

##replacing Nan with 0.0
DF_PTD['PTD']=DF_PTD['PTD'].fillna(0.0)

DF_PTD[DF_PTD['MARKET']=="TSA"]

,MARKET,PTD,PER LE,ABS PTD vs PER LE,PER PTD vs PER LE
29,TSA,0.0,NaN,NaN,NaN


In [0]:
#Add FY Replan
DF_YTD_REPLAN = pd.read_excel(REPLAN_FILE_NAME,parse_cols = "A:R",skiprows = 8,names=['Brand_Seg','Seg','Brand','MARKET','KPI','P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12','P13'])
DF_YTD_REPLAN['MARKET'] = DF_YTD_REPLAN['MARKET'].str.strip()
DF_YTD_REPLAN['KPI'] = DF_YTD_REPLAN['KPI'].str.strip()

#Drop columns and select only 3rd party NSV
DF_YTD_REPLAN = DF_YTD_REPLAN.loc[~DF_YTD_REPLAN['MARKET'].isnull()]
DF_YTD_REPLAN = DF_YTD_REPLAN.loc[DF_YTD_REPLAN['KPI']=='MS.Net Sales 3rd Pty']
DF_YTD_REPLAN = DF_YTD_REPLAN.drop(['Brand_Seg','Seg','Brand','KPI'], axis = 1)
DF_YTD_REPLAN['YTD REPLAN'] = DF_YTD_REPLAN.iloc[:, 1:COL_NUMBER_CURR_PERIOD+1].sum(axis=1)
DF_YTD_REPLAN = DF_YTD_REPLAN[['MARKET','YTD REPLAN']]


BRAND_FILE_PERIOD = LOOKUP_DF.loc[LOOKUP_DF['DATE']<=CURR_DATE, 'BRAND_FILE_PERIOD'].tail(1).values[0]
PERIOD=    LOOKUP_DF.loc[LOOKUP_DF['DATE']<=CURR_DATE, 'GMARSPINY'].tail(1).values[0]

if  PERIOD<=2:
  PTD_YEAR=PTD_YEAR-1
  
print(PTD_YEAR)

#5 and #6. YTD and YTD Replan
DF_YTD = pd.read_csv(CURR_PERIOD_BRAND_FILE_NAME)

DF_YTD = DF_YTD.loc[(DF_YTD['KPI']=='Net Sales 3rd Pty') & (DF_YTD['PERIOD']==BRAND_FILE_PERIOD) & (DF_YTD['YEAR']==PTD_YEAR) & (DF_YTD['Time Range'].isin(['YTD_ACT','YTD_REPLAN']))]
DF_YTD = DF_YTD[['KIND Markets Helios','Time Range','Value']]
DF_YTD.columns = ['Market','Time Range','Value']
DF_YTD = DF_YTD.pivot(index='Market', columns='Time Range', values = 'Value').reset_index()

DF_YTD.columns = ['MARKET','YTD','YTD REPLAN']
DF_YTD = DF_YTD[['MARKET','YTD']]
DF_YTD.loc[DF_YTD["MARKET"]=='MK Fundco Unit', "MARKET"] = 'ROW TSA'
DF_YTD.loc[DF_YTD["MARKET"]=='Gulf', "MARKET"] = 'Gulf Market'


#Merge YTD with YTD Replan
DF_YTD = pd.merge(DF_YTD,DF_YTD_REPLAN, how='left')

new_entry = {'MARKET': 'AIME CEIPA Mkt','PTD':0.0, 'PER LE': 0.0,'ABS PTD vs PER LE':0.0,'PER PTD vs PER LE':0.0}
DF_PTD.loc[len(DF_PTD)]=new_entry


# #Merge YTD with PTD
DF_YTD = pd.merge(DF_PTD,DF_YTD,how='left')

###Adjustment for france by adding AIME CEIPA Mkt
DF_YTD.loc[DF_YTD['MARKET']=='AIME CEIPA Mkt','MARKET']='France'
DF_YTD = DF_YTD.groupby(by=['MARKET'], as_index=False).agg('sum')

DF_YTD['YTD LAST P'] = DF_YTD['YTD']
DF_YTD['YTD'] =  DF_YTD['YTD'] + DF_YTD['PTD']

#7. Difference $ YTD vs YTD Replan
DF_YTD['ABS YTD vs YTD REPLAN'] = DF_YTD['YTD'] - DF_YTD['YTD REPLAN'] 

#8. % Difference % YTD vs YTD Replan
DF_YTD['PER YTD vs YTD REPLAN'] = (DF_YTD['YTD'] / DF_YTD['YTD REPLAN']) 
DF_YTD['PER YTD vs YTD REPLAN'] = DF_YTD['PER YTD vs YTD REPLAN'].round(2)
DF_YTD = DF_YTD.replace(-np.Inf,np.nan)
DF_YTD = DF_YTD.replace(np.Inf,np.nan)

print('YTD SUM', DF_YTD['YTD'].sum())
print('YTD REPLAN SUM', DF_YTD['YTD REPLAN'].sum())

# DF_YTD.MARKET.unique()

2020
YTD SUM 70.1056498915
YTD REPLAN SUM 182.907205177

In [0]:
# DF_YTD.loc[(DF_YTD['KPI']=='Net Sales 3rd Pty') & (DF_YTD['PERIOD']==BRAND_FILE_PERIOD) & (DF_YTD['YEAR']==PTD_YEAR) & (DF_YTD['Time Range'].isin(['YTD_ACT','YTD_REPLAN']))]

In [0]:
# Add FY Replan
DF_REPLAN = pd.read_excel(REPLAN_FILE_NAME,parse_cols = "A:R",skiprows = 8,names=['Brand_Seg','Seg','Brand','MARKET','KPI','P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12','P13'])
DF_REPLAN['MARKET'] = DF_REPLAN['MARKET'].str.strip()
DF_REPLAN['KPI'] = DF_REPLAN['KPI'].str.strip()

#Drop columns and select only 3rd party NSV
DF_REPLAN = DF_REPLAN.loc[~DF_REPLAN['MARKET'].isnull()]
DF_REPLAN = DF_REPLAN.loc[DF_REPLAN['KPI']=='MS.Net Sales 3rd Pty']
DF_REPLAN = DF_REPLAN.drop(['Brand_Seg','Seg','Brand','KPI'], axis = 1)
DF_REPLAN['FY Replan'] = DF_REPLAN.iloc[:, 1:14].sum(axis=1)
DF_REPLAN = DF_REPLAN[['MARKET','FY Replan']]
#DF_REPLAN['FY Replan'] = DF_REPLAN['FY Replan'].round(2)

#Standartize market names across YEE File to match helios names
DF_REPLAN.loc[DF_REPLAN['MARKET']=='Russia/Belarus/CEA','MARKET'] = 'KIND Russia Mkt'
DF_REPLAN.loc[DF_REPLAN['MARKET']=='Gulf', 'MARKET'] = 'Gulf Market'
DF_REPLAN.loc[DF_REPLAN['MARKET']=='Hong Kong', 'MARKET'] = 'Hong Kong Mkt'


DF_REPLAN = pd.merge(DF_YTD,DF_REPLAN,how='left') #Merge with previous DF

In [0]:
#Add FY Replan
DF_REPLAN = pd.read_excel(REPLAN_FILE_NAME,parse_cols = "A:R",skiprows = 8,names=['Brand_Seg','Seg','Brand','MARKET','KPI','P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12','P13'])
DF_REPLAN['MARKET'] = DF_REPLAN['MARKET'].str.strip()
DF_REPLAN['KPI'] = DF_REPLAN['KPI'].str.strip()

#Drop columns and select only 3rd party NSV
DF_REPLAN = DF_REPLAN.loc[~DF_REPLAN['MARKET'].isnull()]
DF_REPLAN = DF_REPLAN.loc[DF_REPLAN['KPI']=='MS.Net Sales 3rd Pty']
DF_REPLAN = DF_REPLAN.drop(['Brand_Seg','Seg','Brand','KPI'], axis = 1)
DF_REPLAN['FY Replan'] = DF_REPLAN.iloc[:, 1:14].sum(axis=1)
DF_REPLAN = DF_REPLAN[['MARKET','FY Replan']]
#DF_REPLAN['FY Replan'] = DF_REPLAN['FY Replan'].round(2)

#Standartize market names across YEE File to match helios names
DF_REPLAN.loc[DF_REPLAN['MARKET']=='Russia/Belarus/CEA','MARKET'] = 'KIND Russia Mkt'
DF_REPLAN.loc[DF_REPLAN['MARKET']=='Gulf', 'MARKET'] = 'Gulf Market'
DF_REPLAN.loc[DF_REPLAN['MARKET']=='Hong Kong', 'MARKET'] = 'Hong Kong Mkt'



DF_REPLAN = pd.merge(DF_YTD,DF_REPLAN,how='left') #Merge with previous DF


#Add FY LE
CURR_PER_LE = pd.read_excel(CURR_PERIOD_YEE_FILE_NAME,parse_cols="A,P:AB" ,skiprows = 3)
CURR_PER_LE=CURR_PER_LE.iloc[:33,:]

CURR_PER_LE = CURR_PER_LE.rename(columns = {'$M 2019':'MARKET'})
CURR_PER_LE.dropna(subset =['MARKET'], inplace = True)
cols=[i for i in CURR_PER_LE.columns if i not in ["MARKET"]]
for col in cols:
    CURR_PER_LE[col]=pd.to_numeric(CURR_PER_LE[col])

CURR_PER_LE['FY LE'] = CURR_PER_LE.sum(axis=1) 
CURR_PER_LE = CURR_PER_LE[['MARKET','FY LE']]
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='UK', 'MARKET'] = 'KIND International Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='USA', 'MARKET'] = 'KIND USA Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='Hong Kong', 'MARKET'] = 'Hong Kong Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=="DESIGNATED CAT.","MARKET"] = 'KIND USA Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=="KIND USA","MARKET"] = 'KIND USA Mkt'

#Standartize market names across YEE File to match helios names
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='Russia/Belarus/CEA','MARKET'] = 'KIND Russia Mkt'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='Gulf', 'MARKET'] = 'Gulf Market'
CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='Hong Kong', 'MARKET'] = 'Hong Kong Mkt'

CURR_PER_LE.loc[CURR_PER_LE['MARKET']=='Central Europe', 'MARKET'] = 'Romania' #adjustment for Central Europe to map it with Romania in Replan file



DF_REPLAN = pd.merge(DF_REPLAN,CURR_PER_LE,how='left')

print('FY LE SUM',DF_REPLAN['FY LE'].sum())
print('FY REPLAN SUM',DF_REPLAN['FY Replan'].sum())

DF_REPLAN.loc[DF_REPLAN['MARKET']=='KIND Corporate Office Unit', 'FY Replan'] = 0.0

#Add YTG
DF_REPLAN['YTG vs FY REPLAN'] = DF_REPLAN['FY Replan'] - DF_REPLAN['YTD']


#Add % 2019(FY) Replan
DF_REPLAN['PER FY REPLAN'] = DF_REPLAN['YTD'] / DF_REPLAN['FY Replan']

# DF_REPLAN[DF_REPLAN['MARKET']=="TSA"]

FY LE SUM 119.167174807
FY REPLAN SUM 200.836945021

In [0]:
#Add YTG vs LE	Difference b/w YTG and LE	FY LE - YTD
DF_REPLAN['YTG vs LE'] = DF_REPLAN['FY LE'] - DF_REPLAN['YTD']

#Add % 2019(FY) LE
DF_REPLAN['PER FY LE'] = DF_REPLAN['YTD'] / DF_REPLAN['FY LE']

In [0]:
#15 $ LE vs. Replan- Difference b/w (FY) LE and (FY)Replan	- FY LE - FY Replan
DF_REPLAN['ABS LE vs. Replan'] = DF_REPLAN['FY LE'] - DF_REPLAN['FY Replan']


#16 % LE vs. Replan  - % difference b/w (FY) LE and (FY)Replan	 - (FY LE - FY Replan) / FY Replan
DF_REPLAN['PER LE vs. Replan'] = (DF_REPLAN['ABS LE vs. Replan'] / DF_REPLAN['FY Replan'])

In [0]:
DF_REPLAN.head(2)

,MARKET,PTD,PER LE,ABS PTD vs PER LE,PER PTD vs PER LE,YTD,YTD REPLAN,YTD LAST P,ABS YTD vs YTD REPLAN,PER YTD vs YTD REPLAN,FY Replan,FY LE,YTG vs FY REPLAN,PER FY REPLAN,YTG vs LE,PER FY LE,ABS LE vs. Replan,PER LE vs. Replan
0,Australia,0.086381,0.320046,-0.233665,0.27,0.599388,4.104014,0.513007,-3.504626,0.15,4.776170,2.109723,4.176782,0.125496,1.510334,0.284108,-2.666448,-0.558282
1,Austria,0.025000,0.074036,-0.049036,0.34,0.525412,1.492810,0.500412,-0.967398,0.35,1.636208,0.758680,1.110795,0.321116,0.233267,0.692535,-0.877528,-0.536318


In [0]:
ALL_MARKETS_DF = DF_REPLAN[['MARKET', 
                      'PTD', 'PER LE', 'ABS PTD vs PER LE', 'PER PTD vs PER LE',
                      'YTD', 'YTD REPLAN', 'YTD LAST P', 'ABS YTD vs YTD REPLAN','PER YTD vs YTD REPLAN',
                      'YTG vs FY REPLAN','FY Replan','PER FY REPLAN',
                      'YTG vs LE','FY LE','PER FY LE',
                      'ABS LE vs. Replan','PER LE vs. Replan']]

ALL_MARKETS_DF = ALL_MARKETS_DF.rename(columns = {'MARKET':'KIND Markets Helios'})

In [0]:
#Replace all infinity and Nan with  0
ALL_MARKETS_DF = ALL_MARKETS_DF.replace(-np.Inf,np.nan)
ALL_MARKETS_DF = ALL_MARKETS_DF.replace(np.Inf,np.nan)

In [0]:
# ALL_MARKETS_DF['KIND Markets Helios'].unique()

In [0]:
# ALL_MARKETS_DF['KIND Markets Helios'].unique()

In [0]:
# KIND_INTERNAL_MAPPING['KIND Markets Helios'].unique()

In [0]:
#Read mapping file and edit
KIND_INTERNAL_MAPPING = pd.read_excel(INTERNAL_MAPPING_FILE_NAME, skiprows = 2)
KIND_INTERNAL_MAPPING = KIND_INTERNAL_MAPPING[['KIND Markets Helios','KIND Markets ','KIND Clusters','KIND Regions']]
KIND_INTERNAL_MAPPING= KIND_INTERNAL_MAPPING.drop_duplicates()

#Map to output dataframe
ALL_MARKETS_DF = pd.merge(ALL_MARKETS_DF,KIND_INTERNAL_MAPPING,how='left')

In [0]:
# ALL_MARKETS_DF.columns

In [0]:
ALL_MARKETS_DF['KIND Markets Helios'].unique()

Out[ 85 ]: array(['Australia', 'Austria', 'Belgium', 'Canada', 'Czech', 'DMEA',
 'France', 'Gulf Market', 'Hong Kong Mkt', 'Hungary', 'Iberia',
 'Ireland Market', 'Israel Mkt', 'Italy', 'Japan', 'KIND China Mkt',
 'KIND Germany Mkt', 'KIND International Mkt', 'KIND Russia Mkt',
 'KIND USA Mkt', 'Levant & Maghreb Mkt', 'Mexico', 'Netherlands',
 'Nordics', 'Poland', 'ROW TSA', 'Romania', 'Saudi Arabia',
 'Slovakia', 'Switzerland', 'TSA', 'Taiwan'], dtype=object)

In [0]:
# ALL_MARKETS_DF.loc[len(ALL_MARKETS_DF)]=['TSA',0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,'TSA','ROW TSA','ROW']

In [0]:
#Add period and year columns 
ALL_MARKETS_DF["YEAR"] = PTD_YEAR
ALL_MARKETS_DF["PERIOD"] = "P" + str(PTD_PERIOD)
ALL_MARKETS_DF["WEEK"] = PTD_WEEK

ALL_MARKETS_DF["R1C1"] = "PTD " + "P" + str(PTD_PERIOD) + ' ' + str(PTD_WEEK)
ALL_MARKETS_DF["R1C2"] = "P" + str(PTD_PERIOD) + ' LE'
ALL_MARKETS_DF["R1C3"] = '$ PTD vs ' + "P" + str(PTD_PERIOD) + ' LE'
ALL_MARKETS_DF["R1C4"] = '% PTD vs ' + "P" + str(PTD_PERIOD) + ' LE'

ALL_MARKETS_DF["R2C1"] = "YTD " + "P" + str(PTD_PERIOD) + ' ' +str(PTD_WEEK)
ALL_MARKETS_DF["R2C2"] = "YTD " + "P" + str(PTD_PERIOD) + ' REPLAN'

In [0]:
# ADD cluster, region and market to ROW
ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='TSA', 'KIND Markets '] = 'TSA'
ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='TSA', 'KIND Clusters'] = 'ROW TSA'
ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='TSA', 'KIND Regions'] = 'ROW'

ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='DMEA', 'KIND Clusters']= 'ROW TSA'
ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='DMEA', 'KIND Regions']= 'ROW'
ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='DMEA', 'KIND Markets ']= 'TSA'



In [0]:
ALL_MARKETS_DF['KIND Regions'].unique()

Out[ 89 ]: array(['UK/Ireland/AUS', 'ROW', 'NA - DC', 'China & North East', nan], dtype=object)

In [0]:
# ALL_MARKETS_DF['YEAR'].unique()

In [0]:
# ALL_MARKETS_DF[ALL_MARKETS_DF['KIND Markets Helios']=="TSA"]['FY LE']

In [0]:
# UK_ADJ = pd.read_excel('/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2019/UK_ADJ.xlsx')
# UK_ADJ = UK_ADJ.loc[UK_ADJ['Period']==PTD_PERIOD]
# UK_ADJ.drop(columns=['Period'], inplace=True)
# UK_ADJ

In [0]:
# UK_ROW = ALL_MARKETS_DF.loc[ALL_MARKETS_DF['KIND Markets Helios']=='KIND International Mkt'].tail(1)
# UK_ROW = UK_ROW.iloc[:,18:]
# UK_ADJ = pd.concat([UK_ADJ.reset_index(drop=True), UK_ROW.reset_index(drop=True)], axis=1)

In [0]:
# UK_ADJ

In [0]:
# #ADD cluster, region and market to ROW
# UK_ADJ['KIND Markets '] = 'UK Adjustment'
# UK_ADJ['KIND Clusters'] = 'UK Adjustment'
# UK_ADJ['KIND Regions'] = 'UK Adjustment'

# UK_ADJ['FY LE']=-0.31

# UK_ADJ

In [0]:
# UK_ADJ.iloc[0,21:] = ALL_MARKETS_DF.iloc[len(ALL_MARKETS_DF)-1,21:]

In [0]:
# ALL_MARKETS_DF = ALL_MARKETS_DF.append(UK_ADJ, sort=False)

## New metrics

In [0]:
NSV_KIND_FILE='/dbfs/mnt/RAW/FILES/KIND_360_GLOBAL/WEEKLY TRACKER/NSV/2020/NSV KIND.xlsx'

In [0]:
#reading the NSV KIND file
MKT_NSV_PTD = pd.read_excel(NSV_KIND_FILE,parse_cols="A:N,P:AB" ,skiprows = 3)

#selecting the required rows
MKT_NSV_PTD=MKT_NSV_PTD.iloc[:33,:]

MKT_NSV_PTD = MKT_NSV_PTD.rename(columns = {'$M 2019':'MARKET'})
MKT_NSV_PTD.dropna(subset =['MARKET'], inplace = True)

cols=[i for i in MKT_NSV_PTD.columns if i not in ["MARKET"]]
for col in cols:
    MKT_NSV_PTD[col]=pd.to_numeric(MKT_NSV_PTD[col])

#Remove total rows 
MKT_NSV_PTD = MKT_NSV_PTD.loc[~MKT_NSV_PTD['MARKET'].isin(['KIND TOTAL','ROW','UK/IRE','CHINA/NEA','ROW (excl. TSA)'])]

#Standartize market names across YEE File to match helios names
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=='UK', 'MARKET'] = 'KIND International Mkt'
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=='Hong Kong', 'MARKET'] = 'Hong Kong Mkt'
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=='USA', 'MARKET'] = 'KIND USA Mkt'
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=="DESIGNATED CAT.","MARKET"] = 'KIND USA Mkt'
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=="KIND USA","MARKET"] = 'KIND USA Mkt'
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=="Gulf","MARKET"] = 'Gulf Market'
MKT_NSV_PTD.loc[MKT_NSV_PTD['MARKET']=="Central Europe","MARKET"] = 'Romania'

In [0]:
##Adding MAT sum 13 periods
START_PER_MAT=PTD_PERIOD
END_PER_MAT=START_PER_MAT+13
MKT_NSV_PTD['MAT']=MKT_NSV_PTD.iloc[:,START_PER_MAT:END_PER_MAT].sum(axis=1)
# MKT_NSV_PTD.head()

# ##Adding MAT LE sum 13 periods
START_PER_LE=PTD_PERIOD+1
END_PER_LE=START_PER_LE+13
MKT_NSV_PTD['MAT LE']=MKT_NSV_PTD.iloc[:,START_PER_LE:END_PER_LE].sum(axis=1)
# MKT_NSV_PTD.head()

# ##Adding the column for runrate
START_PER_RR=PTD_PERIOD+10
END_PER_RR=START_PER_RR+3
MKT_NSV_PTD['Last 3 periods runrate']=MKT_NSV_PTD.iloc[:,START_PER_RR:END_PER_RR].mean(axis=1)
# MKT_NSV_PTD.head()

## creating a column for annualised runrate
MKT_NSV_PTD['Last 3 periods annualized runrate']=MKT_NSV_PTD['Last 3 periods runrate']*13
# MKT_NSV_PTD.head()

MKT_NSV_PTD=MKT_NSV_PTD.rename(columns={"MARKET":"KIND Markets Helios"})
MKT_NSV_PTD=MKT_NSV_PTD[["KIND Markets Helios",'MAT','MAT LE','Last 3 periods runrate','Last 3 periods annualized runrate']]

In [0]:
# # ##TSA not included in calculation for these metrics
# MKT_NSV_PTD.loc[MKT_NSV_PTD['KIND Markets Helios']=="TSA","Last 3 periods runrate"]=0.0
# MKT_NSV_PTD.loc[MKT_NSV_PTD['KIND Markets Helios']=="TSA","Last 3 periods annualized runrate"]=0.0
MKT_NSV_PTD.loc[MKT_NSV_PTD['KIND Markets Helios']=="TSA","MAT"]=0.0
MKT_NSV_PTD.loc[MKT_NSV_PTD['KIND Markets Helios']=="TSA","MAT LE"]=0.0

In [0]:
# MKT_NSV_PTD[MKT_NSV_PTD['KIND Markets Helios']=="TSA"]

In [0]:
FINAL_ALL_MARKETS=pd.merge(ALL_MARKETS_DF,MKT_NSV_PTD,how='left')
# FINAL_ALL_MARKETS

print("Total MAT",FINAL_ALL_MARKETS['MAT'].sum())
print("Total MAT LE",FINAL_ALL_MARKETS['MAT LE'].sum())
print("Total Runrate",FINAL_ALL_MARKETS['Last 3 periods runrate'].sum())
print("Total annualized runrate",FINAL_ALL_MARKETS['Last 3 periods annualized runrate'].sum())

Total MAT 105.541842448
Total MAT LE 112.291613559
Total Runrate 13.3696965631
Total annualized runrate 173.80605532

In [0]:
# FINAL_ALL_MARKETS.columns

In [0]:
#Defining the schema to convert the Pandas dataframe into a spark dataframe
ALL_MARKETS_DF_SCHEMA = StructType([ StructField("KIND Markets Helios", StringType(), True)\

							  ,StructField('PTD', DoubleType(), True)\
#                               ,StructField('PTD NSV Actual', DoubleType(), True)\
							  ,StructField("PER LE", DoubleType(), True)\
							  ,StructField("$ PTD vs PER LE", DoubleType(), True)\
							  ,StructField("% PTD vs PER LE", DoubleType(), True)\
							  
							  ,StructField("YTD", DoubleType(), True)\
							  ,StructField("YTD REPLAN", DoubleType(), True)\
							  ,StructField("YTD LAST P", DoubleType(), True)\
							  ,StructField("$ YTD vs Replan", DoubleType(), True)\
							  ,StructField("% YTD vs Replan", DoubleType(), True)\
							  
							  ,StructField("YTG vs FY Replan", DoubleType(), True)\
							  ,StructField("FY Replan",DoubleType(),True)\
							  ,StructField("% FY Replan",DoubleType(),True)\
							  
							  ,StructField("YTG vs LE",DoubleType(),True)\
							  ,StructField("FY LE",DoubleType(),True)\
							  ,StructField("% FY LE",DoubleType(),True)\
							  
							  ,StructField("$ LE vs Replan",DoubleType(),True)\
							  ,StructField("% LE vs replan", DoubleType(), True)\
                                    
                               ,StructField("KIND Markets", StringType(), True)\
                              ,StructField("KIND Clusters", StringType(), True)\
                              ,StructField("KIND Regions", StringType(), True)\
                              
							  ,StructField("YEAR", IntegerType(), True)\
                              ,StructField("PERIOD", StringType(), True)\
							  ,StructField("WEEK", StringType(), True)\
                                    
                              ,StructField("R1C1", StringType(), True)\
							  ,StructField("R1C2", StringType(), True)\
							  ,StructField("R1C3", StringType(), True)\
							  ,StructField("R1C4", StringType(), True)\
                              
							  ,StructField("R2C1", StringType(), True)\
							  ,StructField("R2C2", StringType(), True)\
       

                              ,StructField("MAT", DoubleType(), True)\
							  ,StructField("MAT LE", DoubleType(), True)\
                              ,StructField("Last 3 periods runrate", DoubleType(), True)\
							  ,StructField("Last 3 periods annualized runrate", DoubleType(), True)])

#Creating a spark dataframe
ALL_MARKETS_OUTPUT_TABLE = spark.createDataFrame(FINAL_ALL_MARKETS,schema = ALL_MARKETS_DF_SCHEMA)

In [0]:
#treating NAN values
columns=ALL_MARKETS_OUTPUT_TABLE.columns
for column in columns:
    ALL_MARKETS_OUTPUT_TABLE = ALL_MARKETS_OUTPUT_TABLE.withColumn(column,F.when(F.isnan(F.col(column)),0).when(F.isnull(F.col(column)),0).otherwise(F.col(column)))

In [0]:
from pyspark.sql.functions import col
ALL_MARKETS_OUTPUT_TABLE=ALL_MARKETS_OUTPUT_TABLE.filter(col("KIND Regions")!="Center")

In [0]:
import time, datetime
from time import strftime
from datetime import datetime

In [0]:
ALL_MARKETS_OUTPUT_TABLE = ALL_MARKETS_OUTPUT_TABLE.withColumn("Datestamp",lit(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')))

In [0]:
ALL_MARKETS_OUTPUT_TABLE=ALL_MARKETS_OUTPUT_TABLE.withColumn("Datestamp", F.from_utc_timestamp(F.col("Datestamp"), "America/New_York"))\
                                                   .withColumn("Datestamp",F.col("Datestamp").cast(StringType()))

In [0]:
# display(ALL_MARKETS_OUTPUT_TABLE)

In [0]:
# Write Final dataframe
ALL_MARKETS_OUTPUT_TABLE.coalesce(1).write.mode('overwrite').option("nullValue", "").option("treatEmptyValuesAsNulls", "false").csv('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/WEEKLY TRACKER/ALL_MARKETS_OUTPUT_TABLE1.csv',header=True)
CSV_DIR = 'dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/WEEKLY TRACKER/'
FILES = [str(x.name) for x in dbutils.fs.ls('dbfs:/mnt/OUTPUT/KIND_360_GLOBAL/GLOBAL_DASHBOARD/WEEKLY TRACKER/ALL_MARKETS_OUTPUT_TABLE1.csv')]
for i in range(len(FILES)):
  if(FILES[i].startswith('part')):
          dbutils.fs.cp(CSV_DIR  + 'ALL_MARKETS_OUTPUT_TABLE1.csv/' + FILES[i],CSV_DIR + FILES[i])
          dbutils.fs.rm(CSV_DIR+'ALL_MARKETS_OUTPUT_TABLE1.csv' , True)
          dbutils.fs.mv(CSV_DIR + FILES[i], CSV_DIR+'ALL_MARKETS_OUTPUT_TABLE1.csv')